# Project 2- H5N1 Outbreak Prediction (Classification Model)

### *Research Question:*

#### Did a H5N1 outbreak occur at a specific location and time?

In [1062]:
# Initial imports
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
# Needed for decision tree visualization
import pydotplus
from IPython.display import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler


# Data Preparation & Cleaning

In [1063]:
# Read the datasets

wildbirds = pd.read_csv("../Data/hpai-wild-birds.csv")
wildbirds = wildbirds.rename(columns={'Date Detected': 'Outbreak Date'})
wildbirds_df = wildbirds[wildbirds['HPAI Strain'].str.contains(r'\bH5N1\b|\bH5\b', na=False)]
wildbirds_df.head()

State      County Collection Date Outbreak Date HPAI Strain  \
0  Alabama   Limestone        2/5/2022     2/23/2022     EA H5N1   
1  Alabama  Montgomery      10/29/2022     11/7/2022  EA/AM H5N1   
2  Alabama  Montgomery      10/29/2022     11/7/2022  EA/AM H5N1   
3  Alabama  Montgomery      10/29/2022     11/7/2022  EA/AM H5N1   
4  Alabama  Montgomery      10/29/2022     11/7/2022  EA/AM H5N1   

      Bird Species WOAH Classification      Sampling Method Submitting Agency  
0  American wigeon           Wild bird       Hunter harvest              NWDP  
1    Black vulture           Wild bird  Morbidity/Mortality              NWDP  
2    Black vulture           Wild bird  Morbidity/Mortality              NWDP  
3    Black vulture           Wild bird  Morbidity/Mortality              NWDP  
4    Black vulture           Wild bird  Morbidity/Mortality              NWDP

In [1064]:
# Clean the dataset by dropping unimportant columns.

wildbirds_df = wildbirds_df.drop(columns = ['Submitting Agency','Sampling Method','Collection Date'])
#wildbirds_df=wildbirds_df.dropna(subset='Outbreak Date')
wildbirds_df.head()

State      County Outbreak Date HPAI Strain     Bird Species  \
0  Alabama   Limestone     2/23/2022     EA H5N1  American wigeon   
1  Alabama  Montgomery     11/7/2022  EA/AM H5N1    Black vulture   
2  Alabama  Montgomery     11/7/2022  EA/AM H5N1    Black vulture   
3  Alabama  Montgomery     11/7/2022  EA/AM H5N1    Black vulture   
4  Alabama  Montgomery     11/7/2022  EA/AM H5N1    Black vulture   

  WOAH Classification  
0           Wild bird  
1           Wild bird  
2           Wild bird  
3           Wild bird  
4           Wild bird

In [1065]:
# Function to convert Date to Day, Month and Year

def convert_date(date):
    return pd.Series([date.year, date.month, date.day], index=["year", "month", "day"])


In [1066]:
wildbirds_df["Outbreak Date"] = pd.to_datetime(wildbirds_df["Outbreak Date"],errors='coerce')
wildbirds_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11614 entries, 0 to 11626
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                11614 non-null  object        
 1   County               11614 non-null  object        
 2   Outbreak Date        11591 non-null  datetime64[ns]
 3   HPAI Strain          11614 non-null  object        
 4   Bird Species         11614 non-null  object        
 5   WOAH Classification  11614 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 635.1+ KB


In [1067]:
# Convert Wild life Outbreak date to Day, Year and Month

#wildbirds_df[['Year','Month','Day']]=wildbirds_df['Outbreak Date'].apply(convert_date)
wildbirds_df.head()

State      County Outbreak Date HPAI Strain     Bird Species  \
0  Alabama   Limestone    2022-02-23     EA H5N1  American wigeon   
1  Alabama  Montgomery    2022-11-07  EA/AM H5N1    Black vulture   
2  Alabama  Montgomery    2022-11-07  EA/AM H5N1    Black vulture   
3  Alabama  Montgomery    2022-11-07  EA/AM H5N1    Black vulture   
4  Alabama  Montgomery    2022-11-07  EA/AM H5N1    Black vulture   

  WOAH Classification  
0           Wild bird  
1           Wild bird  
2           Wild bird  
3           Wild bird  
4           Wild bird

In [1068]:
# Read US counties dataset
us_county_fips=pd.read_csv("../Data/US_Counties.csv")
us_county_fips=us_county_fips.rename(columns={'state_name':'State',"county":"County","county_fips":"FIPS Code","lat":"Latitude","lng":"Longitude"})
us_county_fips=us_county_fips.drop(columns=['state_id','population'])

us_county_fips.head()

State   County  FIPS Code  Latitude  Longitude
0  Alabama  Autauga       1001   32.5349   -86.6427
1  Alabama  Baldwin       1003   30.7277   -87.7226
2  Alabama  Barbour       1005   31.8696   -85.3932
3  Alabama     Bibb       1007   32.9986   -87.1265
4  Alabama   Blount       1009   33.9809   -86.5674

In [1069]:
us_county_fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      3144 non-null   object 
 1   County     3144 non-null   object 
 2   FIPS Code  3144 non-null   int64  
 3   Latitude   3144 non-null   float64
 4   Longitude  3144 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 122.9+ KB


In [1070]:
# Read the weather dataset for adding temperature and Precipitation features

weather_df=pd.read_csv("../Data/weather_cleaned.csv")
weather_df.head()

State    County  Temp  Precipitation
0  Alabama  Autauga   65.9          52.43
1  Alabama  Baldwin   68.7          60.71
2  Alabama  Barbour   66.4          55.97
3  Alabama     Bibb   64.5          46.88
4  Alabama   Blount   63.1          47.63

In [1071]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   State          3107 non-null   object 
 1   County         3107 non-null   object 
 2   Temp           3107 non-null   float64
 3   Precipitation  3107 non-null   float64
dtypes: float64(2), object(2)
memory usage: 97.2+ KB


In [1072]:
# Reading the mammals outbreak datasets
mammal_df = pd.read_csv("../Data/HPAI Detections in Mammals.csv")
mammal_df = mammal_df.rename(columns={'Date Detected':'Outbreak Date','Species':'Mammal Species'})
#mammal_df = mammal_df[mammal_df['HPAI Strain'].str.contains('H5N1', na=False)]
mammal_df["Outbreak Date"] = pd.to_datetime(mammal_df["Outbreak Date"],errors='coerce')
mammal_df=mammal_df.drop(columns=['Date Collected'])
mammal_df.head()

State        County Outbreak Date HPAI Strain Mammal Species
0      Oregon     Multnomah    2025-02-10     EA H5N1   Domestic cat
1     Montana      Flathead    2025-02-10  EA AM H5N1   Domestic cat
2  California     San Mateo    2025-02-05       EA H5   Domestic cat
3    Colorado       Douglas    2025-01-31     EA H5N1   Domestic cat
4  California  Contra Costa    2025-01-24       EA H5   Domestic cat

In [1073]:
# Read mammal df data types
mammal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   State           493 non-null    object        
 1   County          493 non-null    object        
 2   Outbreak Date   493 non-null    datetime64[ns]
 3   HPAI Strain     493 non-null    object        
 4   Mammal Species  493 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 19.4+ KB


In [1074]:

# Reading he backyardflock datasets

backyardflock = pd.read_csv("../Data/backyard_flock_old.csv")
backyardflock = backyardflock.rename(columns={'Outbreak Date': 'Outbreak Date'})
backyardflock["Outbreak Date"] = pd.to_datetime(backyardflock["Outbreak Date"],errors='coerce')
#backyardflock=backyardflock.drop(columns=['FullGeoName','FIPS Codes'])
backyardflock.head()

County           State Outbreak Date  \
0       Ottawa        Michigan    2024-12-31   
1    Riverside      California    2024-12-31   
2  Spartanburg  South Carolina    2024-12-31   
3        Butte      California    2024-12-31   
4        Miner    South Dakota    2024-12-31   

                            Flock Type  Flock Size  
0          Commercial Turkey Meat Bird       29400  
1           Commercial Table Egg Layer      181300  
2  Commercial Upland Gamebird Producer         920  
3                     WOAH Non-Poultry          70  
4                         WOAH Poultry        1500

In [1075]:
backyardflock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554 entries, 0 to 1553
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   County         1554 non-null   object        
 1   State          1554 non-null   object        
 2   Outbreak Date  1554 non-null   datetime64[ns]
 3   Flock Type     1554 non-null   object        
 4   Flock Size     1554 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 60.8+ KB


In [1076]:
# Combine us county fips df with wild birds

merge_df1=wildbirds_df.merge(us_county_fips,on=['State','County'], how='outer')
#merge_df1=merge_df1.merge(bird_migrate_df,on=['Month','Day','Year'], how='outer')
merge_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13576 entries, 0 to 13575
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                13576 non-null  object        
 1   County               13576 non-null  object        
 2   Outbreak Date        11612 non-null  datetime64[ns]
 3   HPAI Strain          11635 non-null  object        
 4   Bird Species         11635 non-null  object        
 5   WOAH Classification  11635 non-null  object        
 6   FIPS Code            13427 non-null  float64       
 7   Latitude             13427 non-null  float64       
 8   Longitude            13427 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 954.7+ KB


In [1077]:
merge_df1.head()

State   County Outbreak Date HPAI Strain   Bird Species  \
0  Alabama  Autauga           NaT         NaN            NaN   
1  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude  
0                 NaN     1001.0   32.5349   -86.6427  
1           Wild bird     1003.0   30.7277   -87.7226  
2           Wild bird     1003.0   30.7277   -87.7226  
3           Wild bird     1003.0   30.7277   -87.7226  
4           Wild bird     1003.0   30.7277   -87.7226

In [1078]:
# Merge the combined dataframe with backyard flock dataframe
merge_df2=merge_df1.merge(backyardflock,on=['State','County','Outbreak Date'], how='outer')
merge_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15108 entries, 0 to 15107
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                15108 non-null  object        
 1   County               15108 non-null  object        
 2   Outbreak Date        13144 non-null  datetime64[ns]
 3   HPAI Strain          11646 non-null  object        
 4   Bird Species         11646 non-null  object        
 5   WOAH Classification  11646 non-null  object        
 6   FIPS Code            13438 non-null  float64       
 7   Latitude             13438 non-null  float64       
 8   Longitude            13438 non-null  float64       
 9   Flock Type           1585 non-null   object        
 10  Flock Size           1585 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 1.3+ MB


In [1079]:
merge_df2.head()

State   County Outbreak Date HPAI Strain   Bird Species  \
0  Alabama  Autauga           NaT         NaN            NaN   
1  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01  EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude Flock Type  Flock Size  
0                 NaN     1001.0   32.5349   -86.6427        NaN         NaN  
1           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN  
2           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN  
3           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN  
4           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN

In [1080]:
# Merge mammals data to the merged dataframe
merge_df3=merge_df2.merge(mammal_df,on=['State','County','Outbreak Date'], how='outer')
merge_df3.head()

State   County Outbreak Date HPAI Strain_x   Bird Species  \
0  Alabama  Autauga           NaT           NaN            NaN   
1  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude Flock Type  Flock Size  \
0                 NaN     1001.0   32.5349   -86.6427        NaN         NaN   
1           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
2           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
3           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
4           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   

  HPAI Strain_y Mammal Species  
0           NaN            NaN  
1           NaN            NaN  
2           NaN            NaN  
3           NaN            NaN  
4           NaN            NaN

In [1081]:
merge_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16515 entries, 0 to 16514
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                16515 non-null  object        
 1   County               16515 non-null  object        
 2   Outbreak Date        14551 non-null  datetime64[ns]
 3   HPAI Strain_x        12688 non-null  object        
 4   Bird Species         12688 non-null  object        
 5   WOAH Classification  12688 non-null  object        
 6   FIPS Code            14480 non-null  float64       
 7   Latitude             14480 non-null  float64       
 8   Longitude            14480 non-null  float64       
 9   Flock Type           1585 non-null   object        
 10  Flock Size           1585 non-null   float64       
 11  HPAI Strain_y        1543 non-null   object        
 12  Mammal Species       1543 non-null   object        
dtypes: datetime64[ns](1), float64(4

In [1082]:
# Merge weather data to the dataframe
merge_df4=merge_df3.merge(weather_df,on=['State','County'], how='left')
merge_df4.head()

State   County Outbreak Date HPAI Strain_x   Bird Species  \
0  Alabama  Autauga           NaT           NaN            NaN   
1  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude Flock Type  Flock Size  \
0                 NaN     1001.0   32.5349   -86.6427        NaN         NaN   
1           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
2           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
3           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
4           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   

  HPAI Strain_y Mammal Species  Temp  Precipitation  
0           NaN            NaN   NaN            NaN  
1           NaN            NaN   NaN            NaN  
2           NaN            NaN   NaN            NaN  
3           NaN            NaN   NaN            NaN  
4           NaN            NaN   NaN            NaN

In [1083]:
# Fill the missing temperature and precipitation rows with mean values


merge_df4['Temp'] = merge_df4.groupby(['State', 'County'])['Temp'].ffill().bfill()
merge_df4['Precipitation'] = merge_df4.groupby(['State', 'County'])['Precipitation'].ffill().bfill()

    
   
merge_df4.head()

State   County Outbreak Date HPAI Strain_x   Bird Species  \
0  Alabama  Autauga           NaT           NaN            NaN   
1  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude Flock Type  Flock Size  \
0                 NaN     1001.0   32.5349   -86.6427        NaN         NaN   
1           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
2           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
3           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
4           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   

  HPAI Strain_y Mammal Species  Temp  Precipitation  
0           NaN            NaN  58.7          25.03  
1           NaN            NaN  58.7          25.03  
2           NaN            NaN  58.7          25.03  
3           NaN            NaN  58.7          25.03  
4           NaN            NaN  58.7          25.03

In [1084]:
# Display the merged dataframe types
merge_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16515 entries, 0 to 16514
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                16515 non-null  object        
 1   County               16515 non-null  object        
 2   Outbreak Date        14551 non-null  datetime64[ns]
 3   HPAI Strain_x        12688 non-null  object        
 4   Bird Species         12688 non-null  object        
 5   WOAH Classification  12688 non-null  object        
 6   FIPS Code            14480 non-null  float64       
 7   Latitude             14480 non-null  float64       
 8   Longitude            14480 non-null  float64       
 9   Flock Type           1585 non-null   object        
 10  Flock Size           1585 non-null   float64       
 11  HPAI Strain_y        1543 non-null   object        
 12  Mammal Species       1543 non-null   object        
 13  Temp                 15249 non-

### Select Target Variable

#### Define a positive case (Outbreak =1)
    If an H5N1 outbreak was recorded at a given state, county and date, label it as 1.
#### Define a Negative case (Outbreak =0)
    For locations and times where no outbreak was recorded, assign 0.

In [1085]:
# Create target column for Outbreak data

merge_df4[['Year','Month','Day']]=merge_df4['Outbreak Date'].apply(convert_date)
merge_df4['Outbreak Occurred'] = merge_df4['Outbreak Date'].notna().astype(int)
merge_df4.head()

State   County Outbreak Date HPAI Strain_x   Bird Species  \
0  Alabama  Autauga           NaT           NaN            NaN   
1  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude Flock Type  Flock Size  \
0                 NaN     1001.0   32.5349   -86.6427        NaN         NaN   
1           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
2           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
3           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
4           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   

  HPAI Strain_y Mammal Species  Temp  Precipitation    Year  Month  Day  \
0           NaN            NaN  58.7          25.03     NaN    NaN  NaN   
1           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   
2           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   
3           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   
4           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   

   Outbreak Occurred  
0                  0  
1                  1  
2                  1  
3                  1  
4                  1

In [1086]:
# Check the value counts of target column "Outbreak occured"
merge_df4['Outbreak Occurred'].value_counts()

Outbreak Occurred
1    14551
0     1964
Name: count, dtype: int64

In [1087]:
# change the Outbreak date column to year, month and day
#merge_df4[['Year','Month','Day']]=merge_df4['Outbreak Date'].apply(convert_date)

merge_df4.head()

State   County Outbreak Date HPAI Strain_x   Bird Species  \
0  Alabama  Autauga           NaT           NaN            NaN   
1  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
2  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
3  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   
4  Alabama  Baldwin    2023-03-01    EA/AM H5N1  Black Vulture   

  WOAH Classification  FIPS Code  Latitude  Longitude Flock Type  Flock Size  \
0                 NaN     1001.0   32.5349   -86.6427        NaN         NaN   
1           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
2           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
3           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   
4           Wild bird     1003.0   30.7277   -87.7226        NaN         NaN   

  HPAI Strain_y Mammal Species  Temp  Precipitation    Year  Month  Day  \
0           NaN            NaN  58.7          25.03     NaN    NaN  NaN   
1           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   
2           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   
3           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   
4           NaN            NaN  58.7          25.03  2023.0    3.0  1.0   

   Outbreak Occurred  
0                  0  
1                  1  
2                  1  
3                  1  
4                  1

In [1088]:
merge_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16515 entries, 0 to 16514
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                16515 non-null  object        
 1   County               16515 non-null  object        
 2   Outbreak Date        14551 non-null  datetime64[ns]
 3   HPAI Strain_x        12688 non-null  object        
 4   Bird Species         12688 non-null  object        
 5   WOAH Classification  12688 non-null  object        
 6   FIPS Code            14480 non-null  float64       
 7   Latitude             14480 non-null  float64       
 8   Longitude            14480 non-null  float64       
 9   Flock Type           1585 non-null   object        
 10  Flock Size           1585 non-null   float64       
 11  HPAI Strain_y        1543 non-null   object        
 12  Mammal Species       1543 non-null   object        
 13  Temp                 15249 non-

# Data Encoding

In [1089]:
# Encoding the categorical columns to numerical data
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Fit and transform categorical columns
categorical_cols = ['State', 'County', 'HPAI Strain_x','HPAI Strain_y','Flock Type', 'Bird Species', 'WOAH Classification','Mammal Species']

for col in categorical_cols:
    merge_df4[col] = encoder.fit_transform(merge_df4[col])


merge_df4.head()

State  County Outbreak Date  HPAI Strain_x  Bird Species  \
0      0      85           NaT              9           231   
1      0      92    2023-03-01              7            17   
2      0      92    2023-03-01              7            17   
3      0      92    2023-03-01              7            17   
4      0      92    2023-03-01              7            17   

   WOAH Classification  FIPS Code  Latitude  Longitude  Flock Type  \
0                    3     1001.0   32.5349   -86.6427          23   
1                    2     1003.0   30.7277   -87.7226          23   
2                    2     1003.0   30.7277   -87.7226          23   
3                    2     1003.0   30.7277   -87.7226          23   
4                    2     1003.0   30.7277   -87.7226          23   

   Flock Size  HPAI Strain_y  Mammal Species  Temp  Precipitation    Year  \
0         NaN              3              40  58.7          25.03     NaN   
1         NaN              3              40  58.7          25.03  2023.0   
2         NaN              3              40  58.7          25.03  2023.0   
3         NaN              3              40  58.7          25.03  2023.0   
4         NaN              3              40  58.7          25.03  2023.0   

   Month  Day  Outbreak Occurred  
0    NaN  NaN                  0  
1    3.0  1.0                  1  
2    3.0  1.0                  1  
3    3.0  1.0                  1  
4    3.0  1.0                  1

In [1090]:
# dipaly the merged dataframe
merge_df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16515 entries, 0 to 16514
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   State                16515 non-null  int32         
 1   County               16515 non-null  int32         
 2   Outbreak Date        14551 non-null  datetime64[ns]
 3   HPAI Strain_x        16515 non-null  int32         
 4   Bird Species         16515 non-null  int32         
 5   WOAH Classification  16515 non-null  int32         
 6   FIPS Code            14480 non-null  float64       
 7   Latitude             14480 non-null  float64       
 8   Longitude            14480 non-null  float64       
 9   Flock Type           16515 non-null  int32         
 10  Flock Size           1585 non-null   float64       
 11  HPAI Strain_y        16515 non-null  int32         
 12  Mammal Species       16515 non-null  int32         
 13  Temp                 15249 non-

In [1091]:
# Handle missing values

merge_df4=merge_df4.fillna(0) # Replace all NaN with 0
merge_df4.head()

State  County        Outbreak Date  HPAI Strain_x  Bird Species  \
0      0      85                    0              9           231   
1      0      92  2023-03-01 00:00:00              7            17   
2      0      92  2023-03-01 00:00:00              7            17   
3      0      92  2023-03-01 00:00:00              7            17   
4      0      92  2023-03-01 00:00:00              7            17   

   WOAH Classification  FIPS Code  Latitude  Longitude  Flock Type  \
0                    3     1001.0   32.5349   -86.6427          23   
1                    2     1003.0   30.7277   -87.7226          23   
2                    2     1003.0   30.7277   -87.7226          23   
3                    2     1003.0   30.7277   -87.7226          23   
4                    2     1003.0   30.7277   -87.7226          23   

   Flock Size  HPAI Strain_y  Mammal Species  Temp  Precipitation    Year  \
0         0.0              3              40  58.7          25.03     0.0   
1         0.0              3              40  58.7          25.03  2023.0   
2         0.0              3              40  58.7          25.03  2023.0   
3         0.0              3              40  58.7          25.03  2023.0   
4         0.0              3              40  58.7          25.03  2023.0   

   Month  Day  Outbreak Occurred  
0    0.0  0.0                  0  
1    3.0  1.0                  1  
2    3.0  1.0                  1  
3    3.0  1.0                  1  
4    3.0  1.0                  1

In [1092]:
# Rename the strains column names

merge_df4=merge_df4.rename(columns={'HPAI Strain_x': 'HPAI Wildbird Strain', 'HPAI Strain_y': 'HPAI Mammal Strain'})
merge_df4.to_csv("../Data/final_dataset.csv")
# Remove the outbreak date column as it is already convered to Year, Month, Day columns
#merge_df4= merge_df4[merge_df4['Month'] != 0]
encoded_df=merge_df4.drop(columns=['Outbreak Date'])

encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16515 entries, 0 to 16514
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   State                 16515 non-null  int32  
 1   County                16515 non-null  int32  
 2   HPAI Wildbird Strain  16515 non-null  int32  
 3   Bird Species          16515 non-null  int32  
 4   WOAH Classification   16515 non-null  int32  
 5   FIPS Code             16515 non-null  float64
 6   Latitude              16515 non-null  float64
 7   Longitude             16515 non-null  float64
 8   Flock Type            16515 non-null  int32  
 9   Flock Size            16515 non-null  float64
 10  HPAI Mammal Strain    16515 non-null  int32  
 11  Mammal Species        16515 non-null  int32  
 12  Temp                  16515 non-null  float64
 13  Precipitation         16515 non-null  float64
 14  Year                  16515 non-null  float64
 15  Month              

In [1093]:
# Correlation map
correlation_df=encoded_df.drop(columns=['State','County','FIPS Code','WOAH Classification','Day','Year'])
correlation_df.corr()

HPAI Wildbird Strain  Bird Species  Latitude  Longitude  \
HPAI Wildbird Strain              1.000000      0.493929 -0.416914   0.425135   
Bird Species                      0.493929      1.000000 -0.406806   0.400805   
Latitude                         -0.416914     -0.406806  1.000000  -0.889177   
Longitude                         0.425135      0.400805 -0.889177   1.000000   
Flock Type                       -0.286631     -0.300006  0.552432  -0.538997   
Flock Size                        0.091708      0.098447 -0.179690   0.173766   
HPAI Mammal Strain                0.028198     -0.081496  0.185253  -0.104487   
Mammal Species                    0.087536     -0.039050  0.133900  -0.028871   
Temp                              0.015801     -0.000375 -0.116521   0.050573   
Precipitation                     0.009534      0.001360 -0.140214   0.031166   
Month                            -0.305673     -0.279600  0.017407  -0.061487   
Outbreak Occurred                -0.461877     -0.471444 -0.059059   0.063836   

                      Flock Type  Flock Size  HPAI Mammal Strain  \
HPAI Wildbird Strain   -0.286631    0.091708            0.028198   
Bird Species           -0.300006    0.098447           -0.081496   
Latitude                0.552432   -0.179690            0.185253   
Longitude              -0.538997    0.173766           -0.104487   
Flock Type              1.000000   -0.384678           -0.067377   
Flock Size             -0.384678    1.000000            0.022026   
HPAI Mammal Strain     -0.067377    0.022026            1.000000   
Mammal Species         -0.071071    0.023234            0.831428   
Temp                   -0.020875    0.015155            0.041179   
Precipitation           0.016483    0.014001            0.030898   
Month                  -0.027207    0.004561            0.050784   
Outbreak Occurred      -0.087706    0.028672           -0.103688   

                      Mammal Species      Temp  Precipitation     Month  \
HPAI Wildbird Strain        0.087536  0.015801       0.009534 -0.305673   
Bird Species               -0.039050 -0.000375       0.001360 -0.279600   
Latitude                    0.133900 -0.116521      -0.140214  0.017407   
Longitude                  -0.028871  0.050573       0.031166 -0.061487   
Flock Type                 -0.071071 -0.020875       0.016483 -0.027207   
Flock Size                  0.023234  0.015155       0.014001  0.004561   
HPAI Mammal Strain          0.831428  0.041179       0.030898  0.050784   
Mammal Species              1.000000  0.009007      -0.008184  0.054641   
Temp                        0.009007  1.000000       0.753749 -0.008095   
Precipitation              -0.008184  0.753749       1.000000  0.011479   
Month                       0.054641 -0.008095       0.011479  1.000000   
Outbreak Occurred          -0.109373 -0.058322      -0.082530  0.553924   

                      Outbreak Occurred  
HPAI Wildbird Strain          -0.461877  
Bird Species                  -0.471444  
Latitude                      -0.059059  
Longitude                      0.063836  
Flock Type                    -0.087706  
Flock Size                     0.028672  
HPAI Mammal Strain            -0.103688  
Mammal Species                -0.109373  
Temp                          -0.058322  
Precipitation                 -0.082530  
Month                          0.553924  
Outbreak Occurred              1.000000

In [1094]:
# Plot the correlation

plt.figure(figsize=(10, 4))
sns.heatmap(correlation_df.corr(), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

# Key Observations from the Correlation Matrix:

Outbreak Risk has positive correlation with Month (0.59), indicating that the month (likely seasonality) plays a significant role in predicting the outbreaks.

HPAI Mammal Strain and Mammal Species have a high positive correlation (0.891854), which suggests that that specific mammal species are more prone to certain HPAI strains.

Flock Type and Latitude have a moderate positive correlation (0.57), indicating that certain types of flocks might be located at specific latitudes.

The HPAI Strain_x (Wild bird strain) has a moderate positive correlation with the Longitude which means certain bird species are specific to that longitudinal locations. 



In [1095]:
# Plot the month distribution
plt.figure(figsize=(10,4))  
sns.histplot(correlation_df[correlation_df['Month'] != 0]['Month'],bins=12, kde=True,color="skyblue")
plt.xlabel("Month")
plt.ylabel("Frequency")
plt.title("Distribution of Month")

Text(0.5, 1.0, 'Distribution of Month')

# Data Splitting

In [1096]:
# Split the training testing data
X=correlation_df.copy().drop(columns='Outbreak Occurred')
y=correlation_df['Outbreak Occurred']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,test_size=0.3)

In [1097]:
X_train.shape

(11560, 11)

In [1098]:
X_test.shape

(4955, 11)

# Data Scaling

In [1099]:
# Use Standard Scaler for scaling the numerical data

scaler=StandardScaler()
X_train_scaled =scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[ 1.25626845,  1.28674666,  0.16130271, ...,  0.82926009,
         1.51177939, -1.50203502],
       [ 0.64971261,  0.85868242, -0.05746262, ...,  0.19140939,
        -0.04196472, -0.32604282],
       [ 1.25626845,  1.28674666, -2.40523056, ..., -0.24348882,
        -1.02839254,  0.61475094],
       ...,
       [ 1.25626845,  1.28674666,  0.1935368 , ...,  0.82926009,
         1.51177939, -1.50203502],
       [ 1.25626845,  1.28674666, -2.40523056, ...,  0.16821482,
        -0.41488256,  0.84994938],
       [ 0.64971261, -0.06230429,  0.96419446, ..., -3.23558851,
        -1.61928687,  0.61475094]])

# Model and fit the data to Logistic regression

In [1100]:
# Build logistic regression model

from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train_scaled, y_train)


LogisticRegression()

In [1101]:
# Score the model
print(f"Training Data Score: {logistic_regression_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {logistic_regression_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.9985294117647059
Testing Data Score: 0.9989909182643795


In [1102]:
# Make predictions on the test data
predictions = logistic_regression_model.predict(X_test_scaled)

# Create a confusion matrix
print(confusion_matrix(y_test, predictions, labels = [1,0]))

[[4399    0]
 [   5  551]]


In [1103]:
# R2 Score

from sklearn.metrics import r2_score

r2_value = r2_score(y_test, predictions)
print("R2 Score :",r2_value)

R2 Score : 0.989870572285068


In [1104]:
# Create a classification report
print(classification_report(y_test, predictions, labels = [1, 0]))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      4399
           0       1.00      0.99      1.00       556

    accuracy                           1.00      4955
   macro avg       1.00      1.00      1.00      4955
weighted avg       1.00      1.00      1.00      4955



In [1105]:
# Calculate the balanced accuracy score
print(balanced_accuracy_score(y_test, predictions))

0.9955035971223021


# Random Forest model

In [1106]:
# Train a Random Forest model

from sklearn.ensemble import RandomForestClassifier

rf_model=RandomForestClassifier(random_state=1,n_estimators=500).fit(X_train_scaled,y_train)

In [1107]:
# Evaluate the model
print(f'Training Score: {rf_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {rf_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 1.0


In [1108]:
# Get the feature importance array
feature_importances = rf_model.feature_importances_

In [1109]:
# Plot the feature importances
features = sorted(zip(X.columns, feature_importances), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(8,6)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

In [1110]:
# Make predictions on the test data
predictions1 = rf_model.predict(X_test_scaled)

# Create a confusion matrix
print(confusion_matrix(y_test, predictions1, labels = [1,0]))

[[4399    0]
 [   0  556]]


In [1111]:
# Create a classification report
print(classification_report(y_test, predictions1, labels = [1, 0]))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      4399
           0       1.00      1.00      1.00       556

    accuracy                           1.00      4955
   macro avg       1.00      1.00      1.00      4955
weighted avg       1.00      1.00      1.00      4955



### Extra Trees Classifier

In [1112]:
# Import an Extremely Random Trees classifier
from sklearn.ensemble import ExtraTreesClassifier

In [1113]:
# Train the ExtraTreesClassifier model
etc_model = ExtraTreesClassifier(random_state=1).fit(X_train_scaled, y_train)

# Evaluate the model
print(f'Training Score: {etc_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {etc_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 0.9995963673057517


### Gradient Boosting Classifier

In [1114]:
# Import Gradient Boosting classifier
from sklearn.ensemble import GradientBoostingClassifier

In [1115]:
gbc_model=GradientBoostingClassifier(random_state=1).fit(X_train_scaled,y_train)

# Evaluate the model
print(f'Training Score: {gbc_model.score(X_train_scaled, y_train)}')
print(f'Testing Score: {gbc_model.score(X_test_scaled, y_test)}')

Training Score: 1.0
Testing Score: 1.0


## Random Undersampling

In [1116]:
# Instantiate the RandomUnderSampler instance
rus = RandomUnderSampler(random_state=1)

# Fit the data to the model
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)


In [1117]:
# Check the sample counts
y_resampled.value_counts()

Outbreak Occurred
0    1408
1    1408
Name: count, dtype: int64

In [1118]:
# Instantiate an initial RamdonForestClassifier instance
rfc_resample_model = RandomForestClassifier()

# Fit the initial model based the training data
rfc_resample_model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [1119]:
# Make predictions using the model based on the resampled data
y_pred_resampled = rfc_resample_model.predict(X_test)

In [1120]:
# Print classification report for resampled data

print(classification_report(y_test, y_pred_resampled, labels = [1, 0]))
#print(classification_report(y_test, predictions1, labels = [1, 0]))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      4399
           0       1.00      1.00      1.00       556

    accuracy                           1.00      4955
   macro avg       1.00      1.00      1.00      4955
weighted avg       1.00      1.00      1.00      4955

